In [114]:
# Импортируем необходимые библиотеки

import pandas as pd
import numpy as np
import json
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
import random

from tensorflow.keras.preprocessing.text import Tokenizer, one_hot, tokenizer_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from sklearn.model_selection import train_test_split

In [115]:
!ls

drive  sample_data


Подгатавливаем данные

In [116]:
df=pd.read_excel('drive/MyDrive/nlp_data/stock_emotion_excel.xlsx')

In [117]:
df.head(5)

,text1,text2,text3,text4,text5,label,ticker
0,Apple unveils the highly anticipated iPhone 14...,"Apple launches the Apple Car, an electric vehi...",Apple announces record-breaking revenue in its...,"Apple introduces the Apple Studio Display, a h...",Apple expands its services portfolio with the ...,apple,$APPL
1,Norilsk Nickel faces renewed scrutiny over env...,Norilsk Nickel's profits plunge amidst allegat...,Norilsk Nickel hit with hefty fines for violat...,Norilsk Nickel's reputation tarnished by repor...,Norilsk Nickel faces backlash from indigenous ...,nornikel,$GMKN
2,Tesla faces production delays and quality cont...,Tesla recalls thousands of vehicles over safet...,Tesla's Autopilot system under scrutiny after ...,Tesla's CEO Elon Musk faces criticism for cont...,Tesla's market share in China declines amid in...,tesla,$TSLA
3,Yandex faces allegations of data privacy breac...,Yandex's ride-hailing service Yandex.Taxi come...,Yandex's advertising practices draw criticism ...,Yandex's partnership with the Russian governme...,Yandex's autonomous vehicle program faces setb...,yandex,$YNDX
4,RZD (Russian Railways) faces criticism over sa...,RZD announces significant fare increases for p...,RZD's freight transportation division experien...,RZD's high-speed rail project encounters delay...,RZD's financial performance worsens amid econo...,RZD,$RZD


In [118]:
tonal_model = load_model('drive/MyDrive/nlp_data/lstm_tonal_text.h5')
emotion_model=load_model('drive/MyDrive/nlp_data/emotions_classification.h5')

In [119]:
with open('drive/MyDrive/nlp_data/tokenizer_tonal.json') as f:
    data = json.load(f)
    tokenizer_tonal = tokenizer_from_json(data)

with open('drive/MyDrive/nlp_data/tokenizer_emotions.json') as f:
    data = json.load(f)
    tokenizer_emotions = tokenizer_from_json(data)

In [120]:
# создаем функцию обработки текста
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''

    sentence = sen.lower()


    # удаляем пунктуацию и числа
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # удалем одиночные символы
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # удаляем лишние пробелы
    sentence = re.sub(r'\s+', ' ', sentence)



    return sentence

In [121]:
emotions_dict={
    '0':['🥺','😔','🤧'],
    '1':['❤️','🥰','😘'],
    '2':['🙌','🤗','🍀','🤑'],
    '3':['🤩','🎉','🎁','👀'],
    '4':['😥','😬','😱','🐻'],
    '5':['🤬','😤','😠','🚨','🤡','🔞','❗','📉'],
}

In [122]:
# @title
#
# ind=5
# text1=df['text1'][ind]
# text2=df['text2'][ind]
# text3=df['text3'][ind]
# text4=df['text4'][ind]
# text5=df['text5'][ind]

In [123]:
# @title
#
# texts=[text1,text2,text3,text4,text5]

In [124]:
# @title
# #
# clean_texts=[]

# for news in texts:
#   news = preprocess_text(news)
#   clean_texts.append(news)

In [125]:
# @title
# # токенизируем текст с помощью ранее обученного токенизатора
# tonal_tokenized = tokenizer_tonal.texts_to_sequences(clean_texts)
# emotions_tokenized = tokenizer_emotions.texts_to_sequences(clean_texts)

# # изменяем длину до 100 слов
# tonal_texts = pad_sequences(tonal_tokenized, padding='post', maxlen=100)
# emotion_texts = pad_sequences(emotions_tokenized, padding='post', maxlen=30)


In [126]:
# @title
# Пропускаем через модель
# tonal_predict = tonal_model.predict(tonal_texts)

# tonal_predict

In [127]:
# @title
# ищем наиболее подходящий под комментарии текст
# mean_score=float(tonal_predict.mean())
# min_metr=10
# min_index=10
# for i in range(len(tonal_predict)):
#   if abs(mean_score-float(tonal_predict[i]))<min_metr:
#     min_metr=abs(mean_score-float(tonal_predict[i]))
#     min_index=i







In [128]:
# @title
# Пропускаем через модель
# emotion_predict = emotion_model.predict(emotion_texts)

# emotion_predict

In [129]:
# @title
# emotion=emotion_predict[min_index].argmax()
# emotion=random.choice(emotions_dict[f'{emotion}'])

Пишем готовую функцию

In [130]:
def get_result(index):
  text1=df['text1'][index]
  text2=df['text2'][index]
  text3=df['text3'][index]
  text4=df['text4'][index]
  text5=df['text5'][index]

  texts=[text1,text2,text3,text4,text5]

  # очищаем текст
  clean_texts=[]

  for news in texts:
    news = preprocess_text(news)
    clean_texts.append(news)

  # токенизируем текст с помощью ранее обученного токенизатора
  tonal_tokenized = tokenizer_tonal.texts_to_sequences(clean_texts)
  emotions_tokenized = tokenizer_emotions.texts_to_sequences(clean_texts)

  # изменяем длину до 100 слов
  tonal_texts = pad_sequences(tonal_tokenized, padding='post', maxlen=100)
  emotion_texts = pad_sequences(emotions_tokenized, padding='post', maxlen=30)

  # Пропускаем через модель
  tonal_predict = tonal_model.predict(tonal_texts)

  # ищем наиболее подходящий под комментарии текст
  mean_score=float(tonal_predict.mean())
  min_metr=10
  min_index=10
  for i in range(len(tonal_predict)):
    if abs(mean_score-float(tonal_predict[i]))<min_metr:
      min_metr=abs(mean_score-float(tonal_predict[i]))
      min_index=i

  # Пропускаем через модель
  emotion_predict = emotion_model.predict(emotion_texts)

  emotion=emotion_predict[min_index].argmax()
  emotion=random.choice(emotions_dict[f'{emotion}'])

  return emotion + df['ticker'][index]+ ' ' + texts[min_index]

In [131]:
res=[]
for i in range(15):
  res.append(get_result(i))

1/1 [==============================] - 0s 438ms/step


<ipython-input-130-b4192de42032>:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if abs(mean_score-float(tonal_predict[i]))<min_metr:
<ipython-input-130-b4192de42032>:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  min_metr=abs(mean_score-float(tonal_predict[i]))


1/1 [==============================] - 0s 36ms/step


In [132]:
for ans in res:
  print(ans)
  print('---------------------')


🤑$APPL Apple expands its services portfolio with the launch of Apple Fitness+, offering personalized workout routines and fitness tracking integration with Apple Watch.
---------------------
🔞$GMKN Norilsk Nickel faces renewed scrutiny over environmental practices following another major diesel spill in the Arctic, raising concerns about its sustainability efforts.
---------------------
🤬$TSLA Tesla's market share in China declines amid increasing competition from domestic electric vehicle manufacturers and regulatory challenges, posing threats to its growth prospects in the world's largest EV market.
---------------------
😬$YNDX Yandex's ride-hailing service Yandex.Taxi comes under fire for reports of driver exploitation and unsafe working conditions, leading to protests and calls for improved labor practices.
---------------------
📉$RZD RZD announces significant fare increases for passenger rail services, drawing backlash from commuters and advocacy groups over affordability and acce